# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import sys
import math
import numpy as np
from collections import namedtuple

Pose = namedtuple("Pose", ["x", "y", "theta"])
Landmark = namedtuple("Landmark", ["x", "y"])
Measurement = namedtuple("Measurement", ["r", "b"])

def eval_gaussian(x, mean=0, var=1):
  return 1.0/math.sqrt(2*math.pi*var)*math.exp(-0.5*(x-mean)**2/var)

class Simulator:
  def __init__(self):
    self.robot = Pose(0, 0, 0)

    self.motion_noise_covariance = [[1, 0], [0, 1]]
    self.sensor_noise_covariance = [[1, 0], [0, np.deg2rad(5)**2]]

    self.landmarks = [Landmark(-1, -1), Landmark(-1, -1), Landmark(11, 11), Landmark(-1, 11)]

  def reset(self):
    self.robot = Pose(0, 0, 0)

  def simulate_motion(self, delta_p, delta_theta):
    noise = np.random.multivariate_normal([0, 0], self.motion_noise_covariance)
    x = self.robot.x + math.cos(self.robot.theta)*(delta_p + noise[0])
    y = self.robot.y + math.sin(self.robot.theta)*(delta_p + noise[0])
    theta = self.robot.theta + delta_theta + noise[1]
    self.robot = Pose(x, y, theta)

  def simulate_sensing(self):
    measurements = []
    for landmark in self.landmarks:
      noise = np.random.multivariate_normal([0, 0], self.sensor_noise_covariance)

      r = math.hypot(landmark[0]-self.robot.x, landmark[1]-self.robot.y) + noise[0]
      b = math.atan2(landmark[1]-self.robot.y, landmark[0]-self.robot.x) + noise[1]

      measurements.append(Measurement(r, b))

    return measurements

  def stringify(self):
    # For CAT-SOOP internal use only. Please ignore.
    # TODO (Jiayuan):: actually stringify it...
    return 'Simulator()'


def drive_in_a_square(simulator):
  poses = []
  poses.append(simulator.robot)
  for sides in range(4):
    for i in range(100):
      simulator.simulate_motion(.1, 0)
      poses.append(simulator.robot)
    simulator.simulate_motion(0, np.pi/2)
    poses.append(simulator.robot)

  return np.array(poses)

def press(event):
  import matplotlib.pyplot as plt
  if event.key == 'q':
    sys.exit(0)
  plt.close()

def draw_samples(samples):
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots()
  ax.axis([-2, 12, -2, 12])
  fig.canvas.mpl_connect('key_press_event', press)
  ax.plot(samples[:, 0], samples[:, 1], 'g+')
  plt.show()

def draw_trajectories(true_poses, estimated_poses):
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots()
  ax.axis([-20, 20, -20, 20])
  fig.canvas.mpl_connect('key_press_event', press)

  ax.plot(true_poses[:, 0], true_poses[:,1], 'b')
  ax.plot(estimated_poses[:, 0], estimated_poses[:,1], 'r')
  plt.show()




## Problems

### Warmup 1
Let's make sure we know how to resample a set of
weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights. Please generate 100 samples.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.choice.


For reference, our solution is **1** lines of code.

In [ ]:
def warmup_1(samples, weights):
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.09468722, 0.15524672, 0.11023114, 0.03947188, 0.15281643, 0.03731063, 0.14530068, 0.13947055, 0.04645405, 0.07901071], dtype=np.float64)), np.array([4, 6, 6, 4, 4, 6, 4, 8, 9, 3, 7, 4, 5, 9, 0, 0, 0, 7, 7, 7, 9, 7, 4, 7, 1, 6, 1, 9, 4, 4, 2, 7, 4, 5, 0, 6, 6, 6, 9, 6, 2, 4, 6, 0, 6, 6, 1, 1, 2, 3, 5, 4, 9, 1, 1, 1, 6, 2, 4, 1, 1, 1, 6, 1, 1, 3, 7, 1, 7, 1, 9, 4, 9, 6, 7, 0, 2, 1, 2, 1, 2, 4, 0, 6, 5, 2, 4, 0, 5, 9, 2, 6, 1, 6, 2, 1, 5, 0, 7, 0], dtype=np.int64), atol=1e-6)
print('Tests passed.')

### Warmup 2
We are going to be sampling from Gaussian
systems in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.multivariate_normal.


For reference, our solution is **2** lines of code.

In [ ]:
def warmup_2(mean, covariance, num_samples):
  """Sample from a multi-variate Gaussian distribution.

  Args:
    mean: a vector in numpy.
    covariance: a matrix in numpy.
    num_samples: an integer.

  Returns:
    sample: a numpy vector with length num_samples.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(warmup_2(np.array([ 0.35563307, -0.45998405,  0.47038804], dtype=np.float64), np.array([[2.18405112, 1.00414281, 1.49286069],  [1.00414281, 0.58927147, 0.64943064],  [1.49286069, 0.64943064, 1.09813033]], dtype=np.float64), 5), np.array([[-2.35284352, -1.74151879, -1.13022555],  [-2.79906741, -2.68743248, -1.56968391],  [-1.03394876, -1.07404052, -0.54291808],  [-0.41991666, -0.6895003 ,  0.26060185],  [-0.81622825, -0.99467384, -0.2275568 ]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 1
Let us now start by building a motion model for a
quasi-holonomic ground robot.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose accoding to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable
given as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

At each step, please use numpy.random.multivariate_normal to sample the noise.


For reference, our solution is **13** lines of code.

In [ ]:
def motion_model(samples, delta_p, delta_theta, simulator):
  """A motion model.

  Args:
    samples: a list of samples
    delta_p: translation command
    delta_theta: rotation command
    simulator: a Simulator object

  Returns:
    a list of new samples by applying the motion model
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(motion_model(np.array([[ 0.14940791,  0.03336743, -0.01790344],  [-0.08540957,  0.03130677, -0.22279039],  [ 0.08644362,  0.06536186, -0.06476612],  [-0.14543657,  0.22697546,  0.00399318],  [ 0.15327792, -0.01871839,  0.12822574],  [ 0.03781625,  0.01549474, -0.07747392],  [-0.03479121, -0.19807965,  0.01364402],  [ 0.12023798,  0.12302907, -0.03380064],  [-0.1048553 , -0.03023028, -0.12391994],  [ 0.19507754, -0.17062702, -0.04447554]], dtype=np.float64), 1, 0, Simulator()), np.array([[ 2.91301728, -0.01611596,  0.38225377],  [ 1.84442332, -0.40589916,  2.01810281],  [ 2.94798951, -0.12022892, -1.042044  ],  [ 1.8046363 ,  0.2347625 , -0.14736402],  [ 1.0426968 ,  0.09595719,  0.53882424],  [ 1.17842815, -0.07305016,  1.37679959],  [ 1.7260826 , -0.17405276,  0.13531904],  [ 1.5632765 ,  0.07423486,  0.29987369],  [ 2.37009854, -0.33850601, -0.32907821],  [ 1.50684678, -0.22900717, -0.89857128]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Sampling Question 2
Let us now add in a measurement model. Let us assume that
there are four landmarks located at locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
bearing [r, b] from the robot's location to each of these landmarks. Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Given a set of measurements, we can use this model to compute the importance
weights of a set of samples of the robot pose.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.



For reference, our solution is **18** lines of code.

In [ ]:
def compute_weights(samples, measurement, simulator):
  """Compute the importance weights of the samples.

  Args:
    a list of samples
    a measurement vector
    a simulator object

  Returns:
    a list of importance weights
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);assert np.allclose(compute_weights(np.array([[ 0.06536186, -0.25529898,  0.07543629],  [ 0.22697546, -0.0742165 , -0.12691735],  [-0.01871839,  0.00457585,  0.13376022],  [ 0.01549474,  0.14693588,  0.03300091],  [-0.19807965, -0.08877857, -0.03036106],  [ 0.12302907,  0.0156349 ,  0.10492744],  [-0.03023028, -0.03873268, -0.09150351],  [-0.17062702, -0.14200179,  0.17023727],  [-0.04380743, -0.05096522, -0.10932702],  [-0.16138978,  0.07774904, -0.01856509]], dtype=np.float64), [Measurement(r=0.5187470011794195, b=-2.3224308778331726), Measurement(r=0.9034084248042221, b=-2.4592240845320847), Measurement(r=15.528166957765391, b=0.8227771705573576), Measurement(r=11.111878239570428, b=1.6878518665472155)], Simulator()), np.array([0.01711671, 0.04172281, 0.14431346, 0.04618495, 0.05960219, 0.16749519, 0.17919444, 0.15658841, 0.18128191, 0.00649993], dtype=np.float64), atol=1e-6)
print('Tests passed.')

### Main Question
Now let's put everything together. Please write a method
localization() that does the following:

  * Creates a simulator.
  * Initialize a set of samples drawn from a Gaussian distribution with mean
equal to the robot starting position, and a diagonal covariance matrix equal
to diag(1, 1, np.deg2rad(5))^2.
  * Drive the robot along the same pattern as in the drive_in_a_square method,
but on each step, after the command motion do the following:
  * use the motion_model method to update the sample between each simulated
motion,
  * use compute_weights to compute the sample weights from the
measurements,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `diag(.01, .01, np.deg2rad(1))^2`.


For reference, our solution is **55** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
def localization(simulator):
  """Main function for robot localization.

  Args:
    simulator: a Simulator object.

  Returns:
    true_poses
    estimated_poses
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);ret = localization(Simulator())
target = (np.array([[ 0.        ,  0.        ,  0.        ],  [ 0.25494743,  0.        ,  0.37816252],  [ 1.02545828,  0.30611138,  0.50714543],  [ 2.79378167,  1.28862186,  1.98766022],  [ 3.12288925,  0.5454071 ,  3.10467651],  [ 2.39815192,  0.57217377,  1.50261885],  [ 2.37338207,  0.2094225 ,  1.98410033],  [ 2.5727203 , -0.24510131,  1.93153303],  [ 2.67841492, -0.52527692,  2.30158892],  [ 3.78365572, -1.75832857,  2.75252338],  [ 3.09589471, -1.47636902,  1.18189997],  [ 2.54741589, -2.81488821,  3.55299425],  [ 1.65300541, -3.20511824,  3.43788678],  [ 0.8195709 , -3.45954946,  4.46732561],  [ 0.84931723, -3.34060699,  3.55424639],  [ 2.92343582, -2.43257883,  3.57458057],  [ 3.25932063, -2.27731864,  3.3578491 ],  [ 3.60488371, -2.20140122,  3.76111364],  [ 5.10757522, -1.12970251,  2.84833015],  [ 5.68666997, -1.30457134,  3.52830499],  [ 7.28359715, -0.65427557,  3.8319089 ],  [ 8.25494437,  0.14792892,  3.79050809],  [ 8.92235572,  0.65415673,  4.77012131],  [ 8.99472247, -0.59793777,  4.94262573],  [ 8.82207184,  0.13864802,  3.40103833],  [ 7.18595417, -0.29562354,  3.14246569],  [ 6.39552545, -0.29631362,  3.9391378 ],  [ 6.16639148, -0.53108289,  4.14061775],  [ 5.78319217, -1.12660267,  5.25558007],  [ 5.39023502, -0.47576962,  5.58388137],  [ 5.33803133, -0.43186113,  5.96179147],  [ 5.62901807, -0.5287414 ,  6.49283398],  [ 5.45606678, -0.56554114,  7.61673928],  [ 5.27827331, -1.30084512,  9.48904967],  [ 5.52753976, -1.28480224, 11.51252161],  [ 4.86209988, -0.1143154 , 11.06017134],  [ 4.80578441,  0.75626795, 12.40639241],  [ 3.11945219,  1.02836962, 12.81772666],  [ 4.82473449,  1.46626374, 11.40112317],  [ 4.46795822,  2.29723321, 10.40293144],  [ 4.7279644 ,  2.68334846, 10.87896283],  [ 4.85744363,  3.78865833, 11.16273238],  [ 5.01559362,  2.85137654, 10.32360819],  [ 5.20256852,  3.08642929,  8.85128526],  [ 5.03029348,  3.19770299, 11.58361539]], dtype=np.float64), np.array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],  [ 1.88924299e-01,  7.02189515e-02,  1.24333283e-01],  [ 1.73388683e+00,  7.30477049e-01,  1.91624164e+00],  [ 1.49886872e+00,  1.16849301e+00,  1.45132509e+00],  [ 2.16992287e+00,  1.14686303e+00,  2.34076642e+00],  [ 2.54110060e+00,  1.15364693e+00,  2.31201774e+00],  [ 2.67141628e+00,  6.66263418e-01,  1.80645930e+00],  [ 2.26324885e+00, -8.31609243e-01,  3.10253091e+00],  [ 1.90424130e+00, -8.17578646e-01,  3.23290699e+00],  [ 3.78301044e+00, -1.11083229e+00,  2.34290891e+00],  [ 4.15986141e+00, -1.23600840e+00,  1.20732976e+00],  [ 4.49020218e+00, -3.25534508e+00,  3.45027766e+00],  [ 4.11194496e+00, -3.37596306e+00,  4.84982709e+00],  [ 4.55182673e+00, -3.89456268e+00,  5.53031066e+00],  [ 4.24779192e+00, -3.56225240e+00,  5.08619264e+00],  [ 3.96052292e+00, -2.66738453e+00,  5.30538230e+00],  [ 2.63091665e+00, -1.81011607e+00,  4.64575807e+00],  [ 3.40050482e+00, -2.34019671e+00,  4.82118946e+00],  [ 3.79166510e+00, -1.70075682e+00,  1.64368565e+00],  [ 3.96941540e+00, -1.30604921e+00,  4.83622545e-01],  [ 5.95208077e+00, -9.07541910e-01, -1.19824161e-01],  [ 6.24348097e+00, -9.42626769e-01,  1.48845537e-01],  [ 6.47239000e+00, -3.94541345e-03,  5.32071941e+00],  [ 7.09257156e+00, -8.94452221e-01,  4.68865295e+00],  [ 7.16071593e+00, -9.70527657e-02,  4.98051528e+00],  [ 7.57922616e+00, -1.14264278e-01,  5.58873314e+00],  [ 6.43769997e+00, -3.91882555e-01,  4.71666902e+00],  [ 6.77222311e+00, -6.73187918e-01,  6.04738478e+00],  [ 5.80942013e+00, -1.44156014e+00,  8.11183478e+00],  [ 5.59872477e+00, -6.42636638e-01,  8.05807728e+00],  [ 5.47020397e+00,  1.53567163e-02,  7.90629178e+00],  [ 5.70039532e+00,  4.00195173e-02,  9.13386298e+00],  [ 5.23079213e+00, -2.02237736e-01,  9.94616957e+00],  [ 4.93801235e+00, -1.05926826e+00,  1.26200087e+01],  [ 5.53997402e+00, -1.02694921e+00,  1.28664081e+01],  [ 5.44312163e+00, -8.53969781e-01,  1.17999423e+01],  [ 4.65484580e+00, -1.57992379e-02,  1.15849134e+01],  [ 4.44968463e+00,  1.31801699e+00,  1.30585923e+01],  [ 3.95599438e+00,  1.05327793e+00,  1.35510044e+01],  [ 4.47912577e+00,  2.05456548e+00,  1.40066887e+01],  [ 4.56279717e+00,  2.94791905e+00,  1.41569277e+01],  [ 4.58195832e+00,  2.92222821e+00,  1.46592924e+01],  [ 4.96391231e+00,  2.23779449e+00,  1.49053558e+01],  [ 4.80953419e+00,  2.68877575e+00,  1.46429066e+01],  [ 4.68623132e+00,  2.94718245e+00,  1.65240724e+01]], dtype=np.float64))
assert all((np.allclose(x, y, atol=1e-6) for x, y in zip(ret, target)))
print('Tests passed.')